# Fantasy Football Analysis

The purpose of this notebook is to provide some simple analyses of each year's fantasy football league. The data that I use as input is weekly point totals for each team for the regular season (13 weeks). At this point, I don't consider finer grained data like position points per week, etc.

In [2]:
import pandas as pd

data_file = 'data_2019.csv'
data = pd.read_csv(data_file)
data = data.set_index('Team')

### Basic Analysis
Here, we calculate means and standard deviations for each team.

In [15]:
data_mean = data.mean(axis=1)
data_std = data.std(axis=1)
basic_stats = pd.DataFrame({'Weekly Mean':data_mean, 'Weekly Standard Dev': data_std})

In [17]:
basic_stats

,Weekly Mean,Weekly Standard Dev
Team,,
DJ Purple,84.534,16.073686
Eat 4 Dicks Asendorf,91.492,18.485327
Aah Fuckin’ Shitdick,106.666,21.966251
Pubic Faith,79.336,22.936637
Peter’s Team,84.408,24.313768
Jbone!,89.338,22.114680
Zach Attack,99.304,20.257177
THROW IT TO SANDERS,90.462,19.645151
Lambeau Leapers,115.038,12.617615


In [19]:
basic_stats.sort_values(by=['Weekly Mean'], ascending=False)

,Weekly Mean,Weekly Standard Dev
Team,,
Lambeau Leapers,115.038,12.617615
Aah Fuckin’ Shitdick,106.666,21.966251
Zach Attack,99.304,20.257177
Chubby Winners,97.462,23.460426
Rudy Was Offsides,93.358,20.119223
Frank The Tank,93.170,27.517389
Eat 4 Dicks Asendorf,91.492,18.485327
THROW IT TO SANDERS,90.462,19.645151
Jbone!,89.338,22.114680


In [20]:
basic_stats.sort_values(by=['Weekly Standard Dev'], ascending=False)

,Weekly Mean,Weekly Standard Dev
Team,,
Frank The Tank,93.170,27.517389
Peter’s Team,84.408,24.313768
Chubby Winners,97.462,23.460426
Pubic Faith,79.336,22.936637
Jbone!,89.338,22.114680
Aah Fuckin’ Shitdick,106.666,21.966251
Zach Attack,99.304,20.257177
Rudy Was Offsides,93.358,20.119223
THROW IT TO SANDERS,90.462,19.645151


### Schedule Luck Analysis
Here, I calculate how lucky each team was. This is computed on a weekly basis and asks the question "What if my opponent was different this week?" The way that our league works, you are matched up with one opponent and play head-to-head against them for that week. If you score a lot of points, but end up playing the team that scored the most, that's pretty unlucky. If you score hardly any point, but end up playing the team that scored the least points, that's pretty lucky.

To compute a luck metric, I compute the expected value of your wins for each week by comparing your weekly points to every other team's points that week. 
 - If you scored the most points, you would beat every team that you could play and your expected win value is 1. 
 - If you scored the least points, you would lose to every team that you could play and your expected win value is 0.
 - If your score would beat 4 teams, but lose to 7 teams, than your expected win value is 4/11

### Team Luck Analysis
Here I want to evaluate
a team's week-to-week variance. This is a slightly different luck analysis than the schedule analysis above. Instead of assuming that your score for the week is fixed and looking at who you played, this analysis will keep your opponent the same, but look at your scores for other weeks and see if you would have won. This asks the question "What if my player's schedule was different?"
